#### Unions

The `Union` type allows a model attribute to accept different types.

> ##### Info
>
> You may get unexpected coercion with `Union`; see below. Know that you can also make the check slower but stricter by using `Smart Union`.

In [1]:
from uuid import UUID
from typing import Literal, Union
from typing_extensions import Annotated
from pydantic import BaseModel, Field, ValidationError

In [2]:
class User(BaseModel):
    id: Union[int, str, UUID]
    name: str

In [3]:
user_1 = User(id=123, name="John Doe")
print(user_1)

id=123 name='John Doe'


In [4]:
print(user_1.id)

123


In [5]:
user_2 = User(id="1234", name="John Doe")
print(user_2)

id=1234 name='John Doe'


In [6]:
print(user_2.id)

1234


In [7]:
user_3_uuid = UUID("cf57432e-809e-4353-adbd-9d5c0d733868")
user_3 = User(id=user_3_uuid, name="John Doe")
print(user_3)

id=275603287559914445491632874575877060712 name='John Doe'


In [8]:
print(user_3.id)

275603287559914445491632874575877060712


In [9]:
print(user_3_uuid.int)

275603287559914445491632874575877060712


However, as can be seen above, pydantic will attempt to 'match' any of the types defined under `Union` and will use the first one that matches. In the above example the `id` of `user_3` was defined as a `uuid.UUID` class (which is defined under the attribute's `Union` annotation) but as the `uuid.UUID` can be marshalled into an `int` it chose to match against the `int` type and disregarded the other types.

> ##### Warning
>
> `typing.Union` also ignores order when defined, so `Union[int, float] == Union[float, int]` which can lead to unexpected behaviour when combined with matching based on the Union type order inside other type definitions, such as `List` and `Dict` types (because Python treats these definitions as singletons). For example, `Dict[str, Union[int, float]] == Dict[str, Union[float, int]]` with the order based on the first time it was defined. Please note that this can also be `affected by third party libraries` and their internal type definitions and the import orders.

As such, it is recommended that, when defining `Union` annotations, the most specific type is included first and followed by less specific types. In the above example, the `UUID` class should precede the `int` and `str` classes to preclude the unexpected representation.

In [10]:
class RightUser(BaseModel):
    id: Union[UUID, int, str]
    name: str

In [11]:
user_3_uuid = UUID("cf57432e-809e-4353-adbd-9d5c0d733868")
user_3 = RightUser(id=user_3_uuid, name="John Doe")
print(user_3)

id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'


In [12]:
print(user_3.id)

cf57432e-809e-4353-adbd-9d5c0d733868


In [13]:
print(user_3_uuid.int)

275603287559914445491632874575877060712


> ##### Tip
>
> The type `Optional[x]` is a shorthand for `Union[x, None]`. `Optional[x]` can also be used to specify a required field that can take `None` as a value.

##### Discriminated Unions (a.k.a. Tagged Unions)

When `Union` is used with multiple submodels, you sometimes know exactly which submodel needs to be checked and validated and want to enforce this. To do that you can set the same field - let's call it `my_discriminator` - in each of the submodels with a discriminated value, which is one (or many) `Literal` value(s). For your `Union`, you can set the discriminator in its value: `Field(discriminator='my_discriminator')`.

Setting a discriminated union has many benefits:

* validation is faster since it is only attempted against one model.

* only one explicit error is raised in case of failure.

* the generated JSON schema implements the `associated OpenAPI specification`.

In [14]:
class Cat(BaseModel):
    pet_type: Literal["cat"]
    meows: int

In [15]:
class Dog(BaseModel):
    pet_type: Literal["dog"]
    barks: int

In [16]:
class Lizard(BaseModel):
    pet_type: Literal["reptile", "lizard"]
    scales: bool

In [17]:
class Pet(BaseModel):
    pet: Union[Cat, Dog, Lizard] = Field(..., discrimintor="pet_type")
    n: int

In [18]:
print(f"{Pet(pet={'pet_type': 'dog', 'barks': 3.14}, n=1) = }")

Pet(pet={'pet_type': 'dog', 'barks': 3.14}, n=1) = Pet(pet=Dog(pet_type='dog', barks=3), n=1)


In [19]:
try:
    print(f"{Pet(pet={'pet_type': 'dog'}, n=1) = }")
except ValidationError as e:
    print(e)

5 validation errors for Pet
pet -> pet_type
  unexpected value; permitted: 'cat' (type=value_error.const; given=dog; permitted=('cat',))
pet -> meows
  field required (type=value_error.missing)
pet -> barks
  field required (type=value_error.missing)
pet -> pet_type
  unexpected value; permitted: 'reptile', 'lizard' (type=value_error.const; given=dog; permitted=('reptile', 'lizard'))
pet -> scales
  field required (type=value_error.missing)


> ##### Note
>
> Using the `Annotated Fields syntax` can be handy to regroup the `Union` and `discriminator` information. See below for an example!

> Warning
>
> Discriminated unions cannot be used with only a single variant, such as `Union[Cat]`. Python changes `Union[T]` into `T` at interpretation time, so it is not possible for pydantic to distinguish fields of `Union[T]` from `T`.

##### Nested Discriminated Unions

Only one discriminator can be set for a field but sometimes you want to combine multiple discriminators. In this case you can always create "intermediate" models with `__root__` and add your discriminator.

In [20]:
class BlackCat(BaseModel):
    pet_type: Literal["cat"]
    color: Literal["black"]
    black_name: str

In [21]:
class WhiteCat(BaseModel):
    pet_type: Literal["cat"]
    color: Literal["white"]
    white_name: str

In [22]:
# # Can also be written with a custom root type
# class ColoredCat(BaseModel):
#     __root__: Annotated[Union[BlackCat, WhiteCat], Field(discriminator="color")]
ColoredCat = Annotated[Union[BlackCat, WhiteCat], Field(discriminator="color")]

In [23]:
UpgragedPet = Annotated[Union[ColoredCat, Dog], Field(discriminator="pet_type")]

In [24]:
class Model(BaseModel):
    pet: UpgragedPet
    n: int

In [25]:
m = Model(pet={"pet_type": "cat", "color": "black", "black_name": "felix"}, n=1)
print(m)

pet=BlackCat(pet_type='cat', color='black', black_name='felix') n=1


In [26]:
try:
    m = Model(pet={"pet_type": "cat", "color": "red"}, n="1")
    print(m)
except ValidationError as e:
    print(e)

1 validation error for Model
pet -> Union[BlackCat, WhiteCat]
  No match for discriminator 'color' and value 'red' (allowed values: 'black', 'white') (type=value_error.discriminated_union.invalid_discriminator; discriminator_key=color; discriminator_value=red; allowed_values='black', 'white')


In [27]:
try:
    m = Model(pet={"pet_type": "cat", "color": "black"}, n="1")
    print(m)
except ValidationError as e:
    print(e)

1 validation error for Model
pet -> Union[BlackCat, WhiteCat] -> BlackCat -> black_name
  field required (type=value_error.missing)
